In [111]:
import pandas as pd 
import numpy as np
df = pd.read_csv('data/2022-05-27-city24dataset.csv')

In [112]:
df.columns

Index(['Unnamed: 0', 'Üldinfo', 'aadress', 'date', 'ID', 'Seisukord', 'Tube',
       'Magamistube', 'Vannitube', 'Üldpind', 'Rõdu', 'Rõdu pind', 'Korrus',
       'Korruseid kokku', 'Omandiõigus', 'Turvasüsteem', 'Lisaväärtused',
       'Hind', 'Hind / m²', 'Selle objekti kuumakse Luminoris',
       'Tee kodu korda Luminori abil!', 'Hoone', 'Katastritunnus',
       'Energiamärgis', 'Materjal', 'Ehitusaasta', 'Lisaruumid',
       'Tehnosüsteemid', 'Küte', 'Sanitaar', 'Pliit', 'Side', 'Parkimine',
       'Lisainfo', 'Ventilatsioon', 'Köögi pind', 'Lift', 'Katus',
       'Kinnistu number', 'tasuta parkimine', 'Krunt', 'Suve kommunaalid',
       'Talve kommunaalid', 'tasuline parkimine', 'Saadaval',
       'Lisanduvad kommunaalid', 'Tagatisraha', 'KM', 'Sihtotstarve',
       'Parkimiskohti', 'Lisanduvad kommunaalkulud', 'Maakleritasu',
       'Piirangud'],
      dtype='object')

In [113]:
df.drop(['Unnamed: 0','Üldinfo','Tee kodu korda Luminori abil!','Katastritunnus','Lisaväärtused',
         'Lisaruumid','Rõdu pind',
       'Tehnosüsteemid', 'Küte', 'Sanitaar', 'Pliit', 'Side', 'Parkimine',
       'Lisainfo', 'Ventilatsioon', 'Köögi pind', 'Lift', 'Katus',
       'Kinnistu number', 'tasuta parkimine', 'Krunt', 'Suve kommunaalid',
       'Talve kommunaalid', 'tasuline parkimine', 'Saadaval',
       'Lisanduvad kommunaalid', 'Tagatisraha', 'KM', 'Sihtotstarve',
       'Parkimiskohti', 'Lisanduvad kommunaalkulud', 'Maakleritasu','Vannitube',
       'Piirangud','Hind / m²','Turvasüsteem'],axis=1, inplace=True)

In [114]:
df = df[~df['aadress'].isnull()]

In [115]:
df = df[~df['Hind'].isnull()]

In [123]:
df = df[~df['Tube'].isnull()]

In [116]:
df['Rõdu'].fillna("Puudub",inplace=True)

In [117]:
df.isnull().sum()

aadress                               0
date                                  0
ID                                    0
Seisukord                           314
Tube                                  3
Magamistube                         468
Üldpind                               4
Rõdu                                  0
Korrus                               94
Korruseid kokku                      45
Omandiõigus                         296
Hind                                  0
Selle objekti kuumakse Luminoris      0
Hoone                                10
Energiamärgis                       371
Materjal                            379
Ehitusaasta                         363
dtype: int64

In [137]:
df['Üldpind']

0        94.5 m²
1        53.7 m²
2        51.2 m²
3          87 m²
4        65.9 m²
          ...   
2695    115.7 m²
2696       51 m²
2697       76 m²
2698       69 m²
2699     97.3 m²
Name: Üldpind, Length: 2674, dtype: object

In [144]:
df['Üldpind'][0].astype(str).str.replace('\D+', '')

AttributeError: 'str' object has no attribute 'astype'

In [138]:
import re
def clean_int(val):
    return re.sub(r"\D", "", val)


In [132]:
df['Hind'] = df['Hind'].apply(clean_int).apply(pd.to_numeric)

TypeError: expected string or bytes-like object

In [124]:
magamistuba_avg = df.groupby('Tube').mean()['Magamistube']
def fill_magamistuba(Tube,Magamistube):
    '''
    Checks the value of "magamistube", if it is null, replaces it with the avg value for the rooms.
    '''
    if np.isnan(Magamistube):
        return magamistuba_avg[Tube]
    else:
        return Magamistube

In [125]:
df['Magamistube'] = df.apply(lambda x: fill_magamistuba(x['Tube'], x['Magamistube']), axis=1)

C:\Users\krappo\AppData\Local\Temp\1\ipykernel_15096\763284497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Magamistube'] = df.apply(lambda x: fill_magamistuba(x['Tube'], x['Magamistube']), axis=1)


In [129]:
# clean size column
def clean_int(val):
    return re.sub(r"[^0-9.]", "", val)

df['Üldpind'] = df['Üldpind'].apply(clean_int).apply(pd.to_numeric)
df['Üldpind'].describe()


TypeError: expected string or bytes-like object

In [284]:
df = df[~df['Korrus/Korruseid'].isnull()]
df = df[~df['Energiamärgis'].isnull()]

In [287]:
df.drop(['Seisukord','Ehitusaasta','Omandivorm'],axis=1, inplace=True)

In [288]:
df['Korruseid']= df['Korrus/Korruseid'].str.split('/').str[1]


In [289]:
df['Korruseid']= df['Korrus/Korruseid'].str.split('/').str[1]
df['Korrus']=df['Korrus/Korruseid'].str.split('/').str[0]
df.drop(['Korrus/Korruseid'], axis=1,inplace=True)

In [290]:
df['aadress'] = df['aadress'].apply(lambda ad: ad.split('-',1)[1])

In [291]:
df['hind_m2']=df['hind']/df['Üldpind']

In [ ]:
df['tanav_number'] = df['aadress'].str.split(',').str[0].str.strip()
df['linnaosa']= df['aadress'].str.split(',').str[2].str.strip()
df['tanav'] = df['tanav_number'].str.split(r"\d").str[0].str.strip()

#Getting coordinate data
GOOGLE_MAPS_API_URL = "https://maps.googleapis.com/maps/api/geocode/json"

headers = requests.utils.default_headers()
headers.update(
    {
        'User-agent': 'Mozilla/5.0',
    }
)

def coordinates(inp):

    params = {
            'address': inp,
            'key': '##########'
    }

     # Do the request and get the response data
    req = requests.get(GOOGLE_MAPS_API_URL, params=params)
    res = req.json()

    # Use the first result
    result = res['results'][0]

    lat = result['geometry']['location']['lat']
    lon = result['geometry']['location']['lng']
    address = result['formatted_address']
    return pd.Series([address, lat, lon])
df[['address','lat', 'long']]=df['st_address'].apply(coordinates)


In [292]:
df.to_csv('Data/data-cleaned.csv')